In [1]:
cd ..

/Users/rsmith/PycharmProjects/GoT_2019/GoT_2019_Bracket


In [2]:
import json
import random
import numpy as np

from src.bracket import Bracket, find_all_brackets
from src.show_state import get_curr_show_state
from src.utils import CHAR_TIERS

In [3]:
brackets = find_all_brackets()
show_state = get_curr_show_state()
char_names = sorted([
    char for char in CHAR_TIERS.keys()
    if show_state.state[char].week_died() is None
])

In [4]:
def _week_n_death_ok(char_pred, week):
    if char_pred.pred_type == 'LIVES':
        return False
    if char_pred.pred_type == 'DIES_ON':
        return char_pred.pred_val == week
    
    return char_pred.pred_val >= week


def potential_char_preds(bracket, state):
    return {
        char: {
            'name': char,
            'dies_5': bracket.char_preds[char].potential_points(state.state[char]) if _week_n_death_ok(bracket.char_preds[char], 5) else 0,
            'dies_6': bracket.char_preds[char].potential_points(state.state[char]) if _week_n_death_ok(bracket.char_preds[char], 6) else 0,
            'lives': bracket.char_preds[char].potential_points(state.state[char]) if bracket.char_preds[char].pred_type == 'LIVES' else 0
        }
        for char in char_names
    }


def potential_char_preds_vec(potential_preds):
    return np.array([
        [
            potential_preds[char]['dies_5'],
            potential_preds[char]['dies_6'],
            potential_preds[char]['lives']
        ]
        for char in char_names
    ])


def state_vec_str(state_vec):
    return [
        char_names[ndx] + ': ' + (
            'Dies' if state_vec[ndx] == 1 else 'Lives'
        )
        for ndx in range(len(state_vec))
    ]


def state_vec_str_old(state_vec):
    return [
        char_names[ndx] + ': ' + (
            'Dies on ' if state_vec[ndx] != 2 else 'Lives'
        ) + (
            '' if state_vec[ndx] == 2 else (
                'Week 5' if state_vec[ndx] == 0 else 'Week 6'
            )
        )
        for ndx in range(len(state_vec))
    ]


pd_brackets = [
    {
        'name': bracket.name,
        'curr_points': bracket.points(show_state),
        'char_map': potential_char_preds(bracket, show_state),
        'char_vec': potential_char_preds_vec(potential_char_preds(bracket, show_state))
    }
    for bracket in brackets
]

In [5]:
pd_brackets

[{'name': 'Jorah the Explorer',
  'curr_points': 124,
  'char_map': {'Arya Stark': {'name': 'Arya Stark',
    'dies_5': 20,
    'dies_6': 20,
    'lives': 0},
   'Bran Stark': {'name': 'Bran Stark', 'dies_5': 0, 'dies_6': 0, 'lives': 40},
   'Brienne of Tarth': {'name': 'Brienne of Tarth',
    'dies_5': 0,
    'dies_6': 0,
    'lives': 0},
   'Bronn': {'name': 'Bronn', 'dies_5': 0, 'dies_6': 0, 'lives': 10},
   'Daenerys Targaryen': {'name': 'Daenerys Targaryen',
    'dies_5': 0,
    'dies_6': 0,
    'lives': 40},
   'Davos Seaworth': {'name': 'Davos Seaworth',
    'dies_5': 0,
    'dies_6': 0,
    'lives': 20},
   'Drogon': {'name': 'Drogon', 'dies_5': 0, 'dies_6': 30, 'lives': 0},
   'Edmure Tully': {'name': 'Edmure Tully',
    'dies_5': 0,
    'dies_6': 0,
    'lives': 0},
   'Gendry': {'name': 'Gendry', 'dies_5': 0, 'dies_6': 0, 'lives': 20},
   'Ghost': {'name': 'Ghost', 'dies_5': 0, 'dies_6': 15, 'lives': 0},
   'Gilly': {'name': 'Gilly', 'dies_5': 0, 'dies_6': 0, 'lives': 10},
 

In [12]:
def delta_points(char_vec, state_vec):
    delta = np.zeros((len(state_vec), 3))
    delta[np.arange(len(state_vec)), state_vec] = 1
    
    return np.sum(delta * char_vec)

PLACE_POINTS_MAP = {
    0: 140,
    1: 30,
    2: 10
}
PLACE_POINTS_MAP.update({
    ndx: -5 * (ndx - 2)
    for ndx in range(2, 19)
})


def heuristic(brackets, name, state_vec):
    # Money won + placement I guess
    scores = sorted([
            (
                bracket['curr_points'] + delta_points(bracket['char_vec'], state_vec),
                bracket['name']
            )
            for bracket in brackets
        ],
        key=lambda x: -x[0]
    )
    
    bracket_ndx = [
        ndx for ndx in range(len(scores))
        if scores[ndx][1] == name
    ][0]
    
    return PLACE_POINTS_MAP[bracket_ndx], scores[bracket_ndx][0]


def heuristic_2(brackets, name, state_vec):
    # Money won + placement I guess
    scores = sorted([
            (
                bracket['curr_points'] + delta_points(bracket['char_vec'], state_vec),
                bracket['name']
            )
            for bracket in brackets
        ],
        key=lambda x: -x[0]
    )
    
    bracket_ndx = [
        ndx for ndx in range(len(scores))
        if scores[ndx][1] == name
    ][0]
    
    return bracket_ndx, -scores[bracket_ndx][0]


def win_scenarios(
    brackets,
    bracket_name,
    top_n=5,
    return_vector_sets=False,
    
    num_gens=100,
    pool_size=30,
    mutation_selection_prob=0.1,
    mutation_prob=0.05,
    num_parents=4,
    new_children=10,
    new_random=3
):
    def _heuristic(state_vec):
        tmp = heuristic(brackets, bracket_name, state_vec)
        return tmp
    
    top_finishes = {
        140: set(),
        30: set(),
        10: set()
    }
    
    top_scoring_vecs = [
        ((-150, 0), np.zeros(len(char_names))) for _ in range(top_n)
    ]
    
    def store_top_score(new_vec, heur_score):
        """
        Should be run on each update to store the best scores
        that we will be returning
        """
        nonlocal top_scoring_vecs, top_finishes
        if heur_score[0] in top_finishes:
            top_finishes[heur_score[0]].add(tuple(new_vec))
        if heur_score <= top_scoring_vecs[-1][0]:
            return
        for ndx in range(len(top_scoring_vecs)):
            if top_scoring_vecs[ndx][0] == heur_score and np.all(top_scoring_vecs[ndx][1] == new_vec):
                return
            elif top_scoring_vecs[ndx][0] < heur_score:
                top_scoring_vecs.insert(ndx, (heur_score, new_vec))
                top_scoring_vecs = top_scoring_vecs[:-1]
                return
        
        print(heur_score)
        print(top_scoring_vecs)
        raise RuntimeError("Shouldn't Get Here")
    
    def generate_pool(num_to_generate):
        new_vecs = [
            np.random.randint(1, 3, size=len(char_names))
            for _ in range(num_to_generate)
        ]
        return [
            (
                _heuristic(state_vec),
                state_vec
            )
            for state_vec in new_vecs
        ]
    
    def mutate(old_vec):
        """
        Mutates the passed vector into a new vector
        if returns None, do nothing
        """
        if np.random.random() < mutation_selection_prob:
            mutation_ndxs = (
                np.random.random(len(char_names)) < mutation_prob
            ).astype(int)
            return (
                mutation_ndxs *
                np.random.randint(1, 3, size=len(char_names)) +
                (mutation_ndxs * -1 + 1) *
                old_vec
            )
    
    def create_child(parent1, parent2):
        crossover_point = np.random.randint(len(parent1))
        return np.concatenate([
            parent1[:crossover_point], parent2[crossover_point:]
        ])
        
    def create_children(parents):
        children = []
        for _ in range(new_children):
            p1, p2 = random.choices(parents, k=2)
            child_vec = create_child(p1[1], p2[1])
            children.append(
                (_heuristic(child_vec), child_vec)
            )
        
        return children
    
    pool = generate_pool(pool_size)
    
    for gen_ndx in range(num_gens):
        # Storing top scores in vector
        for score, state_vec in pool:
            store_top_score(state_vec, score)
        
        # Ordering pool to be in order of top score
        pool = sorted(pool, key=lambda x: (-x[0][0], -x[0][1]))
        
        # Find best population members to mate
        parents = pool[:num_parents]
        
        # Adding new children
        pool[-new_random:-new_random - new_children] = create_children(parents)
        
        # Mutating vecs
        for ndx in range(len(pool)):
            mutated_vec = mutate(pool[ndx][1])
            if mutated_vec is not None:
                pool[ndx] = (_heuristic(mutated_vec), mutated_vec)
        
        # Adding new random states
        pool[-1:-new_random] = generate_pool(new_random)
    
    if return_vector_sets:
        return top_scoring_vecs, top_finishes
    else:
        return top_scoring_vecs


In [71]:
wins, win_vec_sets = win_scenarios(
    pd_brackets,
    'HotPieForIronThrone2k19',
    num_gens=300,
    top_n=100,
    mutation_prob=0.15,
    return_vector_sets=True
)
w_first_strs = state_vec_str(wins[0][1])
print('Conditions:')
for to_print in w_first_strs:
    print('   ', to_print)

all_diff_ndxs = set()
all_second_place_diff_ndxs = set()
all_third_place_diff_ndxs = set()

for ws in wins:
    char_state_strs = state_vec_str(ws[1])
    diff_ndxs = [
        ndx for ndx in range(len(char_state_strs))
        if char_state_strs[ndx] != w_first_strs[ndx]
    ]
    
    if ws[0][0] == 140:
        all_diff_ndxs.update(diff_ndxs)
    elif ws[0][0] == 30:
        all_second_place_diff_ndxs.update(diff_ndxs)
    elif ws[0][0] == 10:
        all_third_place_diff_ndxs.update(diff_ndxs)
    
    print(ws[0])
    print('Differing Conditions:')
    for ndx in diff_ndxs:
        print('   ', char_state_strs[ndx])
    print('*' * 80)
    print()

Conditions:
    Arya Stark: Lives
    Bran Stark: Lives
    Brienne of Tarth: Lives
    Bronn: Lives
    Daenerys Targaryen: Dies
    Davos Seaworth: Dies
    Drogon: Lives
    Edmure Tully: Lives
    Gendry: Dies
    Ghost: Dies
    Gilly: Dies
    Grey Worm: Dies
    Hot Pie: Lives
    Jon Snow: Dies
    Meera Reed: Lives
    Nymeria: Dies
    Podrick Payne: Lives
    Robin Arryn: Lives
    Sam (child): Lives
    Samwell Tarly: Dies
    Sansa Stark: Dies
    Tormund Giantsbane: Dies
    Tyrion Lannister: Lives
    Yara Greyjoy: Dies
(0, 391.0)
Differing Conditions:
********************************************************************************

(0, 391.0)
Differing Conditions:
    Ghost: Lives
********************************************************************************

(0, 391.0)
Differing Conditions:
    Ghost: Lives
    Podrick Payne: Dies
********************************************************************************

(0, 381.0)
Differing Conditions:
    Bronn: Dies
*******

In [ ]:
for place, diff_ndxs in [
    ('First Place', all_diff_ndxs),
    ('Second Place', all_diff_ndxs.union(all_second_place_diff_ndxs)),
    ('Third Place', all_diff_ndxs.union(all_second_place_diff_ndxs).union(all_third_place_diff_ndxs))
]:
    if len(diff_ndxs) > 0:
        print(place + " - Iron Clad Conditions")
        for ndx in [
            i for i in range(len(w_first_strs))
            if i not in diff_ndxs
        ]:
            print('   ', w_first_strs[ndx])

        print()
        print()

In [63]:
# Maybe do a set of all wins?
# Then some tree building for finding supersets maybe

In [72]:
{
    key: len(win_vec_sets[key])
    for key in win_vec_sets
}

{140: 0, 30: 0, 10: 0}

In [73]:
mean_lives_vec = np.mean(
    np.array(
        [vec for vec in win_vec_sets[140]]
    ) - 1,
    axis=0
)
mean_lives_vec

/Users/rsmith/env/GoT/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/rsmith/env/GoT/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [70]:
print('Percent of {} Winning Scenarios that the Character Lives\n'.format(len(win_vec_sets[140])))
for ndx in range(len(char_names)):
    print("  {:.0f}% {}".format(100 * mean_lives_vec[ndx], char_names[ndx]))

Percent of 4245 Winning Scenarios that the Character Lives

  86% Arya Stark
  96% Bran Stark
  24% Brienne of Tarth
  22% Bronn
  100% Daenerys Targaryen
  50% Davos Seaworth
  81% Drogon
  20% Edmure Tully
  26% Gendry
  77% Ghost
  75% Gilly
  25% Grey Worm
  0% Hot Pie
  94% Jon Snow
  56% Meera Reed
  81% Nymeria
  79% Podrick Payne
  24% Robin Arryn
  76% Sam (child)
  37% Samwell Tarly
  28% Sansa Stark
  54% Tormund Giantsbane
  53% Tyrion Lannister
  35% Yara Greyjoy


In [78]:
for bracket in pd_brackets:
    b_name = bracket['name']
    print(b_name)
    
    wins, win_vec_sets = win_scenarios(
        pd_brackets,
        b_name,
        num_gens=300,
        top_n=100,
        return_vector_sets=True
    )
    
    mean_lives_vec = np.mean(
        np.array(
            [vec for vec in win_vec_sets[140]]
        ) - 1,
        axis=0
    )
    
    if not np.all(np.isnan(mean_lives_vec)):
        char_win_percs = {
            char_names[ndx]: '{:.0f}%'.format(mean_lives_vec[ndx] * 100)
            for ndx in range(len(char_names))
        }
    else:
        char_win_percs = {
            cn: 'N/A'
            for cn in char_names
        }
    
    filename = 'brackets/{}_bracket.json'.format(b_name.replace(' ', '_').lower())
    with open(filename) as tmp_file:
        bracket_json = json.load(tmp_file)
    
    bracket_json['num_wins'] = len(win_vec_sets[140])
    for cn in bracket_json['preds']:
        if cn in char_names:
            bracket_json['preds'][cn]['win_perc'] = char_win_percs[cn]
        else:
            bracket_json['preds'][cn]['win_perc'] = 'N/A'
    
    with open(filename, 'w') as tmp_file:
        json.dump(bracket_json, tmp_file)


Jorah the Explorer
HotPieForIronThrone2k19
DrogonMyBallsAround
MikeRopenes_JE
Dead Last
Grant’s Wildling Bracket
DrogoShouldaWon
Gerk
Podrick luvs 2 lay pipe 
MURP<3DANY'SHOTPIE
Three eyed Brandon 
My Dick Looks Like Tyrion Lannister
Matt dangus 
House Sepulveda 161
TWAT OYB
Ma"DiesOn" Aberer
Snow 2020
Ikindawannafuckhotpie
